# Introduction to data science
Author: Gérard Lichtert

## Introduction
This notebook is to clean data from a csv, it removes unnecesary columns, computes means and saves the processed data to a new csv file found in the output folder.

It will also make a new dataframe containing averages per day per participant and save it to a csv

## Variables you can change
In the following code cell you can change the variables as you need

In [1]:
# * This is a list of headers we want to delete (exluding the ones with _TZ, _RT and _TZ)
# * the headers with _TZ, _RT and _TZ will be removed automatically.
HEADERS_TO_DELETE: list[str] = [
    "STUDY_ID",
    "STUDY_NAME",
    "STUDY_VERSION",
    "SURVEY_ID",
    "TRIGGER",
    "START_END",
    "RAND_PROB",
    "CONTROLLEVRAAG",
    "INTRO",
    "SLOT",
    "BEDANKT",
    "INLEIDING",
]

In [2]:
from obse import obse

obse.process_obse(HEADERS_TO_DELETE)

Files processed successfully!
